In [2]:
import json
import http.client
import time
# import boto3
from datetime import datetime
import pandas as pd

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
# GLOBAL VARIBALE CONTAINING THE LAST ID NUMBER FOR EACH TABLE
country_last_id = 0
state_last_id = 0
location_last_id = 0
time_last_id = 0
forecast_last_id = 0
observation_last_id = 0
fact_last_id = 0

global_variables = pd.DataFrame({"table":["country", "state", "location", "time", "forecast", "obersvation", "fact"], "last_id":[country_last_id,  state_last_id, location_last_id, time_last_id, forecast_last_id, observation_last_id, fact_last_id]})
global_variables_csv = global_variables.to_csv()


In [5]:
global_variables_csv

',table,last_id\n0,country,0\n1,state,0\n2,location,0\n3,time,0\n4,forecast,0\n5,obersvation,0\n6,fact,0\n'

In [144]:
# This list will be imported from the location database

cities = [
    # 'Albury',
    # 'Coffs%20Harbour',
    'Sydney',
    'Broken%20Hill',
    'Goulburn',
    'Dubbo',
    # 'Newcastle',
    'Orange',
    'Wagga%20Wagga',
    'Maitland',
    'Wollongong',
    'Cessnock',
    # 'Griffith',
    # 'Armidale',
    # 'Grafton',
    # 'Queanbeyan',
    # 'Lismore',
    # 'Lithgow',
    # 'Nowra',
    # 'Tamworth',
    # 'Bathurst',
    # 'Ulladulla',
    # 'Taree',
    # 'Batemans Bay',
    # 'Port Macquarie',
    # 'Parkes',
    # 'Ballina',
    # 'Muswellbrook',
    # 'Kempsey',
    # 'Bowral',
    # 'Central Coast',
    # 'Kurri Kurri',
    # 'Singleton',
    # 'Tweed Heads',
    # 'Raymond Terrace',
    # 'Wauchope',
    # 'Mudgee',
    # 'Gunnedah',
    # 'Moree',
    # 'Byron Bay',
    # 'Inverell',
    # 'Casino',
    # 'Cooma',
    # 'Katoomba', # couldn't find on the api explorer
    # 'Leeton',
    # 'Deniliquin',
    # 'Narrabri',
    # 'Yamba',
    # 'Cowra',
    # 'Yass',
    ]

# Need to find a way to get the name of each city from the database!!
# AND get the unique ID of each city

# time_df = pd.DataFrame(columns = ["time_time", "time_date"])
time_df_base = pd.DataFrame(columns = ["time_minutes", "time_hours", "time_day", "time_month", "time_year"])

forecast_df_base = pd.DataFrame(columns = ["time_minutes", "time_hours", "time_day", "time_month", "time_year"])

observation_df_base = pd.DataFrame(columns = ["observation_condition", "observation_temp_c", "observation_daytime", "observation_wind_kph", "observation_gust_kph", "observation_wind_direction", "observation_humidity", "observation_pressure_mb", "observation_cloudy", "observation_UV"])

fact_df_base = pd.DataFrame(columns = ["location_id", "observation_id", "time_id", "forecast_id"])
# if forecast_id is not empty, then the fact is of a forecasted weather event

# ^^^^^^^^^^
# These are the dataframe which each row will be appended to, must use pd.concat([df1, df2])



# Order of operations
1. Generate locations tables from list of locations (cites/countries) can query to extract province (state), timezone, and coordinate data
2. Create time entry using 15 minute intervals until the previous time of running
3. Create obervation table for each time interval (combine into fact table)
4. create forecasted oberservation table (combine into fact table)

In [148]:
# The locations can be added to the database manually using the script below
# find the state id needs to be done, this could be done using AWS tools?

conn = http.client.HTTPSConnection("wft-geo-db.p.rapidapi.com")
headers = {
    'content-type': "application/octet-stream",
    'X-RapidAPI-Key': "4be2272798mshecd2e852194b761p1ea3c8jsn77834570a147",
    'X-RapidAPI-Host': "wft-geo-db.p.rapidapi.com"
}

conn.request("GET", "/v1/geo/adminDivisions", headers=headers)

res = conn.getresponse()
data = res.read()
decodedData = data.decode("utf-8")
decodedData_json = json.loads(decodedData)

state_last_id = 0
location_last_id = 0
country_last_id = 0

location_df = pd.DataFrame(columns = ["state_id", "location_name", "location_lat", "location_long"])
state_df = pd.DataFrame(columns = ["count_id", "state_name"])
country_df = pd.DataFrame(columns = ["country_name"])

In [145]:
# Time is created at the start of the script
current_time = datetime.now()
time_df = pd.DataFrame({"time_minutes":current_time.minute, "time_hours":current_time.hour, "time_day":current_time.day, "time_month":current_time.month, "time_year":current_time.year}, index=[time_last_id+1])
time_last_id += 1

time_df_base = pd.concat([time_df_base, time_df])

for city in cities:
    print(city)
    conn = http.client.HTTPSConnection("api.weatherapi.com")
    conn.request("GET", '/v1/current.json?key=30dc2bf00de44d26a5862046230803&q=' + city + '&aqi=no')
    res = conn.getresponse()
    data = res.read()
    decodedData = data.decode("utf-8")
    decodedData_json = json.loads(decodedData)

    observation_df = pd.DataFrame({"observation_condition":decodedData_json["current"]["condition"]["text"], "observation_temp_c":decodedData_json["current"]["temp_c"], "observation_daytime":decodedData_json["current"]["is_day"], "observation_wind_kph":decodedData_json["current"]["wind_kph"], "observation_gust_kph":decodedData_json["current"]["wind_kph"], "observation_wind_direction":decodedData_json["current"]["wind_dir"], "observation_humidity":decodedData_json["current"]["humidity"], "observation_pressure_mb":decodedData_json["current"]["pressure_mb"], "observation_cloudy":decodedData_json["current"]["cloud"], "observation_UV":decodedData_json["current"]["uv"]}, index = [observation_last_id+1])
    observation_last_id += 1
    observation_df_base = pd.concat([observation_df_base, observation_df])

    fact_df = pd.DataFrame({"location_id":city, "observation_id":observation_last_id, "time_id":time_last_id, "forecast_id":""}, index=[fact_last_id+1])
    fact_last_id += 1
    fact_df_base = pd.concat([fact_df_base, fact_df])


# we need to adjust how the json is read in
# location_df = pd.DataFrame(data = decodedData_json["location"],  index = [location_last_id + 1])

state_df = pd.DataFrame(columns = ["count_id", "state_name"])
country_df = pd.DataFrame(columns = ["country_name"])
location_last_id += 1


Sydney
Broken%20Hill
Goulburn
Dubbo
Orange
Wagga%20Wagga
Maitland
Wollongong
Cessnock


In [146]:
fact_df_base

,location_id,observation_id,time_id,forecast_id
1,Sydney,1,1,
2,Broken%20Hill,2,1,
3,Goulburn,3,1,
4,Dubbo,4,1,
5,Orange,5,1,
6,Wagga%20Wagga,6,1,
7,Maitland,7,1,
8,Wollongong,8,1,
9,Cessnock,9,1,


In [147]:
# use indexlabel in df.to_csv()

observation_df_base.reset_index(inplace=True)
observation_df_base.rename(columns={"index":"observation_id"}, inplace=True)
# observation_df_base.to_csv("obersvation_26042023.csv")
observation_df_base_csv = observation_df_base.to_csv("obersvation_26042023.csv")

time_df_base.reset_index(inplace=True)
time_df_base.rename(columns={"index":"time_id"}, inplace=True)
# time_df_base.to_csv("time_26042023.csv")
time_df_base_csv = time_df_base.to_csv()

fact_df_base.reset_index(inplace=True)
fact_df_base.rename(columns={"index":"fact_id"}, inplace=True)
# fact_df_base.to_csv("fact_26042023.csv")
fact_df_base_csv = fact_df_base.to_csv()

# do this for forecasts as well, do cities first

In [149]:
decodedData_json["current"]
# decodedData_json["location"]

{'last_updated_epoch': 1682568000,
 'last_updated': '2023-04-27 14:00',
 'temp_c': 24.0,
 'temp_f': 75.2,
 'is_day': 1,
 'condition': {'text': 'Partly cloudy',
  'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
  'code': 1003},
 'wind_mph': 10.5,
 'wind_kph': 16.9,
 'wind_degree': 40,
 'wind_dir': 'NE',
 'pressure_mb': 1022.0,
 'pressure_in': 30.18,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'humidity': 54,
 'cloud': 50,
 'feelslike_c': 25.2,
 'feelslike_f': 77.4,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'uv': 6.0,
 'gust_mph': 12.1,
 'gust_kph': 19.4}

In [150]:
# For observation data
# Get id number for location using AWS tools
print(type(decodedData_json["current"]))

forecast_df = pd.DataFrame(columns = ["time_minutes", "time_hours", "time_day", "time_month", "time_year"])
# Same as above, but for forecast data



<class 'dict'>


# Note that: all times in utc!

In [151]:

# location_df["utc"] = location_df["localtime"].apply(pd.to_datetime)
location_df

,state_id,location_name,location_lat,location_long


In [152]:
# location_df['day'] = location_df["utc"].dt.day
# location_df['month'] = location_df["utc"].dt.month
# location_df['year'] = location_df["utc"].dt.year
# location_df['hour'] = location_df["utc"].dt.hour
# location_df['minute'] = location_df["utc"].dt.minute

In [153]:
# location_df = location_df[["name", "region", "country", "lat", "lon"]]

In [154]:
global_variables = pd.DataFrame({"table":["country", "state", "location", "time", "forecast", "obersvation", "fact"], "last_id":[country_last_id,  state_last_id, location_last_id, time_last_id, forecast_last_id, observation_last_id, fact_last_id]})
global_variables

,table,last_id
0,country,0
1,state,0
2,location,0
3,time,1
4,forecast,0
5,obersvation,9
6,fact,9
